# **Differentiable Wonderland: Chapter 5**
goal: reinforce material from Chapter 5 of Alice's Adventures in a Differentiable Wonderland by implementing PyTorch machine learning workflows. 
- CIFAR10 Image Classifier from Deep Learning with PyTorch: A 60 Minute Blitz
- [choose HuggingFace dataset here]

## **Part 1: Replicating a PyTorch Workflow**

[Deep Learning with PyTorch: A 60 Minute Blitz](https://docs.pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)

## **Part 2: Applying PyTorch to a Hugging Face Dataset**

[Working with Hugging Face Datasets](https://huggingface.co/docs/datasets/en/tutorial)